<a href="https://colab.research.google.com/github/kislay960/Data-Science-Practice/blob/main/Transfer_Learning_Sample_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.layers import Dense,Activation,Flatten,Input,Dropout,GlobalAveragePooling2D
from keras.models import Model
from keras.utils import np_utils
import random
import matplotlib.pyplot as plt

In [ ]:
base_path = '/content/drive/MyDrive/images'
dirs = os.listdir(base_path)

In [ ]:
print(dirs)

['horses', 'cats', 'dogs', 'humans']


In [ ]:
image_data = []
labels = []
label_dict = {'cats':0,'dogs':1,'horses':2,'humans':3}

In [ ]:
for ix in dirs:
    path = os.path.join(base_path,ix)
    img_data = os.listdir(path)
    for im in img_data:
        img = image.load_img(os.path.join(path,im),target_size=(224,224))
        img_array = image.img_to_array(img)
        image_data.append(img_array)
        labels.append(label_dict[ix])

In [ ]:
print(len(image_data), len(labels))

808 808


In [ ]:
combined = list(zip(image_data, labels))
random.shuffle(combined)

image_data[:], labels[:] = zip(*combined)

X_train = np.array(image_data)
Y_train = np.array(labels)

print(X_train.shape,Y_train.shape)

(808, 224, 224, 3) (808,)


In [ ]:
Y_train = np_utils.to_categorical(Y_train)
print(X_train.shape,Y_train.shape)

(808, 224, 224, 3) (808, 4)


In [ ]:
model = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256,activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(4,activation='softmax')(d1)

model_new = Model(inputs=model.input, outputs= fc2)
model_new.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
adam = Adam(lr=0.00003)
model_new.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
for ix in range(len(model_new.layers)):
    print(ix, model_new.layers[ix])

0 <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f2318dab2b0>
1 <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7f22e586c320>
2 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f22e586c4e0>
3 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f22e586c9b0>
4 <tensorflow.python.keras.layers.core.Activation object at 0x7f22e586cef0>
5 <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7f22e4fc7128>
6 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f22df161278>
7 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f22df164a90>
8 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f22df18b8d0>
9 <tensorflow.python.keras.layers.core.Activation object at 0x7f22df112780>
10 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f22df112940>
11 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization o

In [ ]:
for ix in range(169):
    model_new.layers[ix].trainable = False
    
model_new.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model_new.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
hist = model_new.fit(X_train,Y_train,
                    shuffle = True,
                    batch_size = 16,
                    epochs = 5,
                    validation_split=0.20
                    )

Epoch 1/5
41/41 [==============================] - 113s 3s/step - loss: 1.3098 - accuracy: 0.4582 - val_loss: 0.5475 - val_accuracy: 0.9012
Epoch 2/5
41/41 [==============================] - 112s 3s/step - loss: 0.6132 - accuracy: 0.7941 - val_loss: 0.2875 - val_accuracy: 0.9321
Epoch 3/5
41/41 [==============================] - 112s 3s/step - loss: 0.3987 - accuracy: 0.8684 - val_loss: 0.2091 - val_accuracy: 0.9568
Epoch 4/5
41/41 [==============================] - 113s 3s/step - loss: 0.2792 - accuracy: 0.9272 - val_loss: 0.1749 - val_accuracy: 0.9630
Epoch 5/5
41/41 [==============================] - 113s 3s/step - loss: 0.1931 - accuracy: 0.9505 - val_loss: 0.1551 - val_accuracy: 0.9691
